# TRABAJO:

### Realice un programa que capture un video y desarrolle un procedimiento de detección de movimiento de un objeto por sustracción de fondo e identifique el objeto en movimiento con un rectángulo ubicado sobre el mismo en todo momento.

In [ ]:
import cv2
import time
import numpy as np
from matplotlib import colors
from matplotlib import pyplot as plt

#-------------------------------------------------------------------------------------------------------------------

#camara = cv2.VideoCapture(0) #Captura de video con la camara del computador
camara = cv2.VideoCapture(1) #Captura de video con la camara del Web LifeCam VX-1000
#camara = cv2.VideoCapture('stres.mp4') #Video ya existente

fondo = None #Obtenemos el primer Frame como "VACIO" obteniendo asi el fondo con el que comparar
#-------------------------------------------------------------------------------------------------------------------
while True:
    #-------------------------------------------------------------------------------------------------------------------
    (grabbed, frame) = camara.read() #leemos con la camara en la variable frame y grabbed
    
    if not grabbed:#Si puse un video , cuando llegue al final cerrara el while
        break
    #-------------------------------------------------------------------------------------------------------------------
    gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Con esto convertimos el frame del video a escala de grises
    gris = cv2.GaussianBlur(gris, (21, 21), 0) #Asemos un suavizado a frame en grises con GaussianBlur
    
    if fondo is None: #Si no capturamos el fondo ( primer frame capturado) , este sera ahora la mascara gris
        fondo = gris
        continue
    #-------------------------------------------------------------------------------------------------------------------
    resta = cv2.absdiff(fondo, gris) #hago la resta entre el primer frame obtenido (fondo) y la mascara de grises
    umbral = cv2.threshold(resta, 25, 255, cv2.THRESH_BINARY)[1]# Aplicamos un umbral ()
    umbral = cv2.dilate(umbral, None, iterations=2)# Ampliamos el umbral para tapar agujeros
    contornosimg = umbral.copy() #hacmeos una copia del umbral para ahora obtener los contornos
    #-------------------------------------------------------------------------------------------------------------------
    contornos,_ = cv2.findContours(contornosimg,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # buscamos el contorno de la imagen findContours

    for c in contornos: # tomamos cada uno de los contornos
        if cv2.contourArea(c) < 20: #eliminamos solo los contornos por debajo de 5 pixeles
            continue #continua el programa en secuencia
        (x, y, w, h) = cv2.boundingRect(c) # obtenemos las dimenciones del Bpunding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)#dibujamos un rectanculo en la capa frame bajo las coordenadas puestas
    #-------------------------------------------------------------------------------------------------------------------   
    cv2.imshow("Camara", frame) #mostramos el video de la camara
    cv2.imshow("Umbral", umbral)#mostramos la mascra del umbral
    cv2.imshow("Resta", resta)#mostramos la mascara de la resta
    cv2.imshow("Contorno", contornosimg)#mostramos los contornos
    #-------------------------------------------------------------------------------------------------------------------   
    tecla = cv2.waitKey(27) & 0xFF
    if tecla == 27: #para salir del while
        break
#-------------------------------------------------------------------------------------------------------------------
camara.release()    #libero la camara para otros programas
cv2.destroyAllWindows() #cierro las ventanas abiertas